<a href="https://colab.research.google.com/github/jsja22/ai/blob/master/seq1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dat_df = pd.read_csv('/content/drive/MyDrive/energydata.csv',sep=',')
dat_df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09


In [ ]:
#전처리
dat_df[8585:8588]

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5
8585,2016-03-11 07:50:00,70,0,20.2,37.160000,16.5,42.430000,20.39,37.500000,19.700000,34.360000,18.000000,46.660000
8586,2016-03-11 08:00:00,280,0,20.2,37.463333,16.6,42.766667,NaN,37.500000,19.700000,34.633333,18.066667,46.193333
8587,2016-03-11 08:10:00,180,10,20.2,37.530000,16.6,42.966667,20.29,37.363333,19.666667,34.723333,18.100000,45.723333


In [ ]:
#결손값 전후 2행 추출
dat_df[8585:8588].fillna(method='ffill')   #fillna(method='ffill') -> 전행값을 입력한다.

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5
8585,2016-03-11 07:50:00,70,0,20.2,37.160000,16.5,42.430000,20.39,37.500000,19.700000,34.360000,18.000000,46.660000
8586,2016-03-11 08:00:00,280,0,20.2,37.463333,16.6,42.766667,20.39,37.500000,19.700000,34.633333,18.066667,46.193333
8587,2016-03-11 08:10:00,180,10,20.2,37.530000,16.6,42.966667,20.29,37.363333,19.666667,34.723333,18.100000,45.723333


In [ ]:
#평균치로 보완하는방법
dat_df = dat_df.interpolate()  #.interpolate() -> 전후 평균치로 보완(기본적으로 선형보완 실시)
dat_df[8585:8588]

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5
8585,2016-03-11 07:50:00,70,0,20.2,37.160000,16.5,42.430000,20.39,37.500000,19.700000,34.360000,18.000000,46.660000
8586,2016-03-11 08:00:00,280,0,20.2,37.463333,16.6,42.766667,20.34,37.500000,19.700000,34.633333,18.066667,46.193333
8587,2016-03-11 08:10:00,180,10,20.2,37.530000,16.6,42.966667,20.29,37.363333,19.666667,34.723333,18.100000,45.723333


In [ ]:
dat_df['date'] = pd.to_datetime(dat_df['date'],format='%Y-%m-%d %H:%M:%S')
dat_df['dif_min'] = dat_df['date'].diff().dt.total_seconds()/60 #diff 한행전값과의 차이 계산
dat_df['dif_min'] = dat_df['dif_min'].fillna(0)  #fillna -> 결손값 Nan 을 0으로 보완 
dat_df['cum_min'] = dat_df['dif_min'].cumsum()  #dif_min의 누적합계 구하기
dat_df[['date','cum_min']].head()

,date,cum_min
0,2016-01-11 17:00:00,0.0
1,2016-01-11 17:10:00,10.0
2,2016-01-11 17:20:00,20.0
3,2016-01-11 17:30:00,30.0
4,2016-01-11 17:40:00,40.0


In [ ]:
#6시간 단위로 각 항목의 통계량을 계산
#평균치 계산

dat_df['cum_6hour'] = (dat_df['cum_min']/360).round(2).astype(int)
print(dat_df['cum_6hour'].unique())

dat_df = dat_df.drop(['date','dif_min','cum_min'],axis=1)
dat_df_mean = dat_df.groupby('cum_6hour').mean()

print(dat_df_mean.shape) #(484, 12)
dat_df_mean.head()

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5
cum_6hour,,,,,,,,,,,,
0,150.000000,31.944444,20.688056,47.345394,20.023148,44.973287,20.111481,46.041875,19.255972,47.379074,17.761597,60.359861
1,81.944444,7.222222,20.858287,45.627708,20.227315,44.286435,20.213588,45.530833,20.740093,46.503634,18.898519,50.261273
2,83.333333,5.555556,19.860463,47.019213,19.140608,44.876698,20.149722,45.269306,19.349259,45.893611,18.163125,50.817454
3,101.111111,0.000000,20.037500,44.843009,19.103377,43.495476,19.907778,44.802407,19.748287,43.536204,17.835370,50.444111
4,78.611111,7.222222,19.947963,43.275394,19.113860,42.606633,19.788704,44.312037,19.555093,42.804491,17.477870,50.732546


In [ ]:
#6시간 단위 표준편차

dat_df_std = dat_df.groupby('cum_6hour').std()

In [ ]:
print(dat_df_std.shape) #(484, 12)
print(dat_df_std.head())


(484, 12)
           Appliances     lights        T1  ...      RH_4        T5      RH_5
cum_6hour                                   ...                              
0          131.366445  13.901582  0.672443  ...  1.282966  0.791008  9.891611
1          101.497908   9.137399  0.460572  ...  0.511257  0.272610  0.339389
2          104.853639   9.085135  0.103157  ...  0.776746  0.178471  1.138170
3           95.999339   0.000000  0.059026  ...  0.622986  0.164863  0.792715
4           83.773599  12.097488  0.151195  ...  0.430826  0.075740  0.247231

[5 rows x 12 columns]


In [ ]:
dat_features = pd.merge(dat_df_mean, dat_df_std,left_index=True,right_index=True)

print(dat_features.shape) #(484, 24)
print(dat_features.head()) #dat_df_mean에는 _x dat_df_std에는 _y로 구분


(484, 24)
           Appliances_x   lights_x       T1_x  ...    RH_4_y      T5_y    RH_5_y
cum_6hour                                      ...                              
0            150.000000  31.944444  20.688056  ...  1.282966  0.791008  9.891611
1             81.944444   7.222222  20.858287  ...  0.511257  0.272610  0.339389
2             83.333333   5.555556  19.860463  ...  0.776746  0.178471  1.138170
3            101.111111   0.000000  20.037500  ...  0.622986  0.164863  0.792715
4             78.611111   7.222222  19.947963  ...  0.430826  0.075740  0.247231

[5 rows x 24 columns]


In [ ]:
#슬라이드 창을 사용한 특징량 추출
tmp = dat_df